In [1]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import unicodedata
import os
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import logging

# Silence pdfminer and pdfplumber logs
logging.getLogger("pdfminer").setLevel(logging.ERROR)
logging.getLogger("pdfplumber").setLevel(logging.ERROR)

In [79]:
# Slugs que siempre queremos ignorar
EXCLUDE_SLUGS = [
    'informacion_personal',
    'documentacion_personal',
    'situacion_personal_actual',
    'composicion_familiar',
    'datos_de_contacto',
    'lugar_de_nacimiento',
    'conceptorespuesta',
    'documento_nacional_de_identidad_cedula_de_identidad',
    'fondos_economicos',
    'educacion',
    'datos_del_empleador',
    'trabajos_empleos',
    'historial_migratorio',
    'declaraciones_medicas',
    'antecedentes_penales',
    'documentos_solicitados'
]


# Añadimos 'ejemplo' y líneas que acaben en ':' a las instrucciones a ignorar
INSTRUCTION_PATTERNS = [
    r'^indicar',         # empieza con 'Indica'
    r'^revisa',         # empieza con 'Revisa'
    r'^este documento', # empieza con 'Este documento'
    r'^por favor',      # empieza con 'Por favor'
    r'^ejemplo',        # empieza con 'EJEMPLO'
    r'^ej',             # empieza con 'Ej:'
    r'^ten',            # empieza con 'Ten'
    r'^\*',             # empieza con '*',
    r'emitió',          # contiene 'emitió' ,
    r'^solamente',      # empieza con 'Solamente' ,
    r'^puedes',	        # empieza con 'Puedes' ,
    r'^incluye'	        # empieza con 'Incluye' ,
    r'^experiencia',	# empieza con 'Experiencia',
    r'^importante'	    # empieza con 'Importante' ,
]

ATTACHMENT_EXTENSIONS = re.compile(r'\.(pdf|jpe?g|rtf|docx)\b', re.IGNORECASE)

def slugify(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    text = text.lower()
    return re.sub(r'[^a-z0-9]+', '_', text).strip('_')


def is_instruction(text: str) -> bool:
    low = text.lower().strip()
    if low.endswith(':'):
        return True
    return any(re.search(pat, low) for pat in INSTRUCTION_PATTERNS)

def sufijos_minuscula(texto):
    # Separa en "palabras" (sean letras mayúsculas) ignorando espacios y caracteres como "/"
    palabras = re.findall(r'[A-ZÑÁÉÍÓÚ]+', texto)
    # Toma la primera letra de cada palabra, pásala a minúscula y únelas
    return ''.join(p[0].lower() for p in palabras)

def clean_question_for_slug(txt: str) -> str:
    """
    Si detecta una pregunta larga con '¿...?' extrae sólo lo que hay entre la última
    abertura '¿' y el signo '?', o bien la última oración que termine en '?'
    """
    q = txt.strip()
    if '¿' in q and '?' in q:
        start = q.rfind('¿')
        end = q.rfind('?')
        q = q[start+1:end]  # quita '¿' y '?'
    elif '?' in q:
        # toma hasta el signo '?'
        part = q[:q.rfind('?')+1]
        # si hay varios puntos, coge lo que viene después del último punto
        if '.' in part:
            q = part.split('.')[-1]
        else:
            q = part
        q = q.strip(' ?')
    # devolver sin signos innecesarios
    return q

def extract_bold_fields_with_labels(pdf_path: str):
    data, labels = {}, {}
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages[1:]:
            # Agrupar caracteres por línea
            lines = {}
            for ch in page.chars:
                y = round(ch['top'], 1)
                lines.setdefault(y, []).append(ch)
            parsed = []
            for y in sorted(lines):
                chars = lines[y]
                txt = ''.join(c['text'] for c in sorted(chars, key=lambda c: c['x0'])).strip()
                fonts = [c['fontname'] for c in chars]
                if any('Bold' in f for f in fonts):
                    style = 'bold'
                elif any('Light' in f for f in fonts):
                    style = 'light'
                else:
                    style = 'normal'
                parsed.append((txt, style))

            suffix = None
            for i, (txt, style) in enumerate(parsed):
                # Detectar encabezado de sección
                if txt.isupper() and style == 'bold':
                    suffix = sufijos_minuscula(txt)

                # Extraer campo en negrita válido
                if style == 'bold' and not is_instruction(txt):
                    # limpiamos la pregunta para slug
                    clean_q = clean_question_for_slug(txt)
                    base = slugify(clean_q)
                    if base in EXCLUDE_SLUGS:
                        continue

                    # Construir slug con sufijo de sección si existe
                    slug = f"{base}_{suffix}" if suffix else base
                    labels[slug] = txt

                    # Acumular todas las líneas hasta el siguiente bold
                    values = []
                    for j in range(i+1, len(parsed)):
                        vtxt, vst = parsed[j]
                        # si encontramos otro bold, interrumpimos
                        if vst == 'bold':
                            break
                        # ignorar light e instrucciones
                        if vst == 'light' or is_instruction(vtxt):
                            continue
                        values.append(vtxt)
                    # unir con '; '
                    data[slug] = '; '.join(values).strip()

    return data, labels

def map_attachment_cell(v):
    lv = str(v).lower().strip()
    if re.search(ATTACHMENT_EXTENSIONS, lv):
        return 'yes'
    if 'no se ha subido ningún archivo' in lv:
        return 'no'
    return lv or pd.NA

def extract_text_lines(pdf_path: str):
    """Read all text lines from PDF once."""
    lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages[1:]:
            text = page.extract_text() or ''
            lines.extend([l.strip() for l in text.split('\n') if l.strip()])
    return lines

def process_parallel_bold(root_folder: str, field_order=None, field_labels=None):
    """
    Process 'parallel' PDFs in root_folder, returning:
      - DataFrame with one row per PDF
      - Updated field_labels dict mapping slug -> original question text
    """
    if field_order is None:
        field_order = []
    if field_labels is None:
        field_labels = {}

    records = []
    entries = os.listdir(root_folder)
    sample_dirs = [d for d in entries if os.path.isdir(os.path.join(root_folder, d))]
    items = sorted(sample_dirs, key=lambda x: int(x) if x.isdigit() else x) if sample_dirs else sorted(entries)

    for item in tqdm(items, desc="Processing"):
        # Locate PDF
        if os.path.isdir(os.path.join(root_folder, item)):
            folder = os.path.join(root_folder, item)
            pdfs = [f for f in os.listdir(folder) if 'parallel' in f.lower() and f.endswith('.pdf')]
            if not pdfs:
                continue
            pdf_path = os.path.join(folder, pdfs[0])
            sample_id = item
        else:
            if not (item.lower().endswith('.pdf') and 'parallel' in item.lower()):
                continue
            pdf_path = os.path.join(root_folder, item)
            sample_id = os.path.splitext(item)[0]

        # Extract fields and labels
        data, labels = extract_bold_fields_with_labels(pdf_path)

        # Read all lines once for fallback and bullet detection
        text_lines = extract_text_lines(pdf_path)

        # Extraer MONTO:
        monto_val = None
        for line in text_lines:
            m = re.search(r'\bMONTO:\s*(.+)', line, re.IGNORECASE)
            if m:
                monto_val = m.group(1).strip()
                break
        if monto_val is not None:
            data['monto'] = monto_val
            # Asegurarnos de incluirlo en el orden/etiquetas
            if 'monto' not in field_order:
                field_order.append('monto')
                field_labels['monto'] = 'MONTO:'

        # Update field_order and labels with new fields
        for slug, original in labels.items():
            if slug not in field_order:
                field_order.append(slug)
                field_labels[slug] = original

        # Fallback: search only missing slugs in text_lines
        for slug in field_order:
            if slug not in data and slug in field_labels:
                label_text = field_labels[slug].lower()
                for idx, line in enumerate(text_lines):
                    if label_text in line.lower():
                        # Take next non-instruction line as value
                        if idx + 1 < len(text_lines) and not is_instruction(text_lines[idx+1]):
                            data[slug] = text_lines[idx+1]
                        break

        # Exclude unwanted slugs
        for slug in EXCLUDE_SLUGS:
            if slug in field_order:
                field_order.remove(slug)
            field_labels.pop(slug, None)

        # Build row in fixed column order
        row = {slug: data.get(slug) for slug in field_order}
        row['sample_id'] = sample_id
        records.append(row)

    df = pd.DataFrame(records, columns=field_order + ['sample_id'])

    # Poner yes or no en ficheros adjuntos
    attachment_cols = [
        col for col, lbl in field_labels.items()
        if re.search(r'adjuntar|copia|adjunto', lbl, re.IGNORECASE)]
    for col in attachment_cols:
        if col in df.columns:
            df[col] = df[col].apply(map_attachment_cell)
        

    # NaN en lugar de cadena vacía:
    df.replace({'': np.nan}, inplace=True)
    df.replace({r'^(?i:none)$': np.nan}, inplace=True)

    # drops any column where *all* values are NaN
    df = df.dropna(axis=1, how='all')

    return df, field_labels


In [80]:

df_all_granted, dic_questions = process_parallel_bold('data/granted/')
df_all_granted['final_visa_status'] = 'granted'

df_all_refused, dic_questions = process_parallel_bold('data/refused/',
                                field_order=list(dic_questions.keys()),
                                field_labels=dic_questions)
df_all_refused['final_visa_status'] = 'refused'

Processing: 100%|██████████| 4/4 [00:35<00:00,  8.98s/it]
/var/folders/f0/dt7lkxv100q68c369j1bzgk00000gn/T/ipykernel_13386/4273993011.py:247: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'': np.nan}, inplace=True)
Processing: 100%|██████████| 4/4 [00:49<00:00, 12.30s/it]
/var/folders/f0/dt7lkxv100q68c369j1bzgk00000gn/T/ipykernel_13386/4273993011.py:247: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'': np.nan}, inplace=True)


In [81]:
df_all_granted.shape

(4, 153)

In [82]:
df_all_refused.shape

(4, 217)

In [83]:
#search_by_field = [i for i in df_all_refused.columns.unique() if '_ip' in i]

In [84]:
#df_all_refused[search_by_field]

In [85]:
#[i for i in df_all_refused.columns.unique() if 'ties' in i]

In [86]:
#df_all_refused[['family_ties_ds']]

In [87]:
#df_all_refused['family_ties_ds'].loc[3]

In [88]:
df_all = pd.concat([df_all_granted,df_all_refused]).reset_index(drop=True)

In [89]:
df_all.shape

(8, 230)

In [92]:
df_all

,monto,nombre_y_apellido_s_tal_y_como_aparece_en_el_pasaporte_ip,estas_actualmente_en_australia_ip,indica_en_que_pais_te_encuentras_actualmente_ip,situacion_legal_en_dicho_pais_ip,adjuntar_documentacion_respaldatoria_de_tu_situacion_legal_ip,has_estado_previamente_en_australia_ip,podras_encontrar_adjunto_en_este_campo_tu_coe_certificate_of_enrolment_dp,confirma_que_los_datos_de_tu_coe_son_correctos_dp,indica_tu_tus_nacionalidad_es_dp,adjuntar_una_copia_del_pasaporte_que_utilizaras_para_aplicar_dp,numero_de_identificacion_dni_dndicdi,pais_de_expedicion_dndicdi,fecha_de_expedicion,fecha_de_caducidad,adjuntar_copia_de_tu_documento_nacional_de_identidad_o_cedula_de_identidad,estado_o_provincia_en_donde_naciste_ldn,pais_de_nacimiento_ldn,cual_es_su_situacion_personal_actual_spa,estas_aplicando_a_un_visado_de_pareja_spa,direccion_completa_del_pais_de_residencia_habitual_aunque_lleves_mucho_tiempo_en_australia_este_ddc,nombre_de_la_calle_y_numero_ddc,ciudad_ddc,provincia_estado_o_region_ddc,codigo_postal_ddc,pais_ddc,numero_telefono_en_tu_pais_de_residencia_habitual,indica_los_miembros_de_tu_familia_cf,indica_el_nombre_y_apellido_s_de_tus_familiares_cf,indica_la_fecha_de_nacimiento_de_tus_familiares_cf,indica_el_pais_de_residencia_de_tus_familiares,deseas_presentarlos_fe,seran_de_un_avalista_fe,explica_con_detalles_la_documentacion_de_fondos_que_vas_a_presentar_fe,adjuntar_evidencia_de_fondos_extractos_bancarios_propios,adjuntar_introductory_letter_of_funds,indica_los_datos_de_los_estudios_mas_altos_completados_que_tengas_fuera_de_australia_e,nombre_del_curso_e,fecha_inicio_e,fecha_fin_e,institucion_escuela_universidad_e,pais_e,adjuntar_evidencia_de_estudios_mas_altos_finalizados,de_australia,estas_actualmente_desempleado_te,nombre_de_la_empresa_sin_abreviaciones_te,tipo_de_industria_te,direccion_completa_de_la_empresa_te,nombre_y_apellidos_del_empleador_te,telefono_de_contacto_del_empleador_con_prefijo_te,fecha_de_inicio_te,cual_es_tu_posicion,posees_otro_trabajo_actualmente,adjuntar_evidencia_de_empleo_actual_o_de_tus_trabajos_actuales_solo_en_caso_de_que_tengas,mas_de_1,tienes_trabajos_previos,nombre_empresa_sin_abreviaciones,tipo_de_industria,direccion_completa_de_la_empresa,pais,nombre_y_apellidos_de_empleador,telefono_de_contacto_del_empleador_con_prefijo,cual_es_la_posicion_que_tenias,fecha_de_inicio,fecha_de_finalizacion,tienes_otro_trabajo_previo,adjuntar_evidencia_de_los_empleos_anteriores,te_han_ofrecido_trabajo_al_regresar_a_tu_pais,tienes_planeado_alguna_de_estas_opciones_a_la_vuelta,tipo_de_estudio_que_realizaras_a_tu_regreso_pais_e_institucion_si_es_que_lo_sabes,adjuntar_evidencia_de_estudios_de_regreso,has_realizado_algun_examen_de_ingles_en_los_ultimos_24_meses_i,has_estudiado_durante_5_anos_completos_en_algun_pais_de_habla_inglesa_i,has_viajado_en_los_ultimos_10_anos_fuera_de_tu_pais_de_residencia_habitual_v,australia_o_para_cualquier_otro_pais_hm,has_tenido_algun_visado_denegado_de_australia_o_de_cualquier_otro_pais_hm,subir_el_documento_del_visado_denegado_hm,mas_tiempo_del_que_le_permitia_le_visado_ya_sea_en_australia_o_en_cualquier_otro_pais_hm,tu_o_alguien_incluido_en_esta_aplicacion_ha_tenido_alguna_vez_un_visado_cancelado_o_dene_hm,gado_ya_sea_para_australia_o_para_cualquier_otro_pais_hm,da_detalles,3_meses_consecutivos_fuera_del_pais_de_su_pasaporte_dm,tienes_alguna_intencion_de_ingresar_en_algun_hospital_de_australia_dm,_dm,tienes_intencion_de_trabajar_en_algun_centro_educativo_con_ninos_dm,has_tenido_o_tienes_lo_siguiente_dm,tienes_intencion_de_recibir_tratamiento_medico_dm,necesitas_alguna_asistencia_de_movilidad_o_cuidados_especiales_dm,tienes_algun_asunto_pendiente_con_la_justicia_ap,has_sido_arrestado_alguna_vez_ap,has_sido_alguna_vez_declarado_culpable_por_un_delito_sexual_ap,tes_penales_que_no_hayan_sido_contemplados_en_las_preguntas_anteriores_ap,entregado_la_documentacion_solicitada_correcta_y_honestamente_d,denegacion_de_mi_visado_d,adjuntar_formulario_de_autorizacion_956a_individual_ds,adjuntar_c

In [90]:
df_all['podras_encontrar_adjunto_en_este_campo_tu_coe_certificate_of_enrolment_dp'].loc[1]

'yes'

In [91]:
df_all[['podras_encontrar_adjunto_en_este_campo_tu_coe_certificate_of_enrolment_dp']]

,podras_encontrar_adjunto_en_este_campo_tu_coe_certificate_of_enrolment_dp
0,yes
1,yes
2,none
3,yes
4,yes
5,yes
6,yes
7,none
